# Naive Bayes (the easy way)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [3]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('/Users/leonardo/tsp/ml/hands_n/DataScience-Python3/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('/Users/leonardo/tsp/ml/hands_n/DataScience-Python3/emails/ham', 'ham'))


Let's have a look at that DataFrame:

In [5]:
data.tail()

,class,message
/Users/leonardo/tsp/ml/hands_n/DataScience-Python3/emails/ham/01037.6b42b5f3d3d9e6293bf24af66b250655,ham,"On Thu, 2002-08-29 at 00:02, Ville Skyttä wrot..."
/Users/leonardo/tsp/ml/hands_n/DataScience-Python3/emails/ham/02056.7bc7703e40a24dda665d4ce7b0cba710,ham,"URL: http://www.newsisfree.com/click/-6,835599..."
/Users/leonardo/tsp/ml/hands_n/DataScience-Python3/emails/ham/01782.278f53b8f65fcd422cb26c5bbe74599d,ham,use Perl Daily Newsletter\n\n\n\nIn this issue...
/Users/leonardo/tsp/ml/hands_n/DataScience-Python3/emails/ham/00043.d2673a72d215cbdd747dc98cde41fbd2,ham,< >\n\n> I downloaded a driver from the nVidia...
/Users/leonardo/tsp/ml/hands_n/DataScience-Python3/emails/ham/00135.bd3bc1c036eab89c9c50cff40958c939,ham,>That Goddess Chick wrote:\n\n>>\n\n>> >Thank...


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [6]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's try it out:

In [8]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

## Activity

Our data set is small, so our spam classifier isn't actually very good. Try running some different test emails through it and see if you get the results you expect.

If you really want to challenge yourself, try applying train/test to this spam classifier - see how well it can predict some subset of the ham and spam emails.

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data)
print(len(train), len(test))

2250 750


In [20]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(train['message'].values)

classifier = MultinomialNB()
targets = train['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
classifier.score(X=vectorizer.transform(train["message"]), y=train["class"])

0.9742222222222222

In [22]:
classifier.score(X=vectorizer.transform(test["message"]), y=test["class"])

0.9546666666666667